<a href="https://colab.research.google.com/github/Xujjjjun2002/Kaggle/blob/main/Kaggle_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 前期准备工作

### 关联google drive

In [ ]:
from google.colab import drive 
drive.mount('/content/drive/') 

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### 更新kaggle api，下载对应的文件并解压缩（暂时没搞明白什么意思）

In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"ulrika","key":"c54a613de021c8e7f66b3c021c318136"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c titanic
!unzip titanic.zip

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 24.8MB/s]
Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


## 从现在开始进入代码的编写

### 导入包和数据

In [ ]:
import torch as to
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from scipy.stats import uniform, randint
from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold #交叉验证
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train_data_all = pd.read_csv("train.csv")
test_data_all = pd.read_csv("test.csv")
train_data_all.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 首先进行数据的预处理工作

#### 先进行描述性统计观察大致情况，统计缺失值

In [ ]:
train_data_all.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
train_data_all.isnull().sum()#缺失值统计

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

####  Pclass的分析和处理

In [ ]:
train_data_all[["Pclass","Survived"]].groupby("Pclass",as_index=False).mean()
# 从上面我们可以看出，Pclass对于生还概率存在影响

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


#### Sex的分析和处理

In [ ]:
train_data_all[['Sex','Survived']].groupby('Sex',as_index=False).mean()#影响比较显著

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [ ]:
#我们把'Sex'表示为num(描述性特征数值化)
train_data_all["Sex"]=train_data_all["Sex"].map({"male":1,"female":2}).astype(int)
test_data_all["Sex"]=test_data_all["Sex"].map({"male":1,"female":2}).astype(int)

#### Embarked的分析和处理

In [ ]:
#首先考虑缺失值，采用众数的方法进行补值
train_data_all["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [ ]:
train_data_all["Embarked"].fillna("S",inplace=True)
train_data_all[["Embarked","Survived"]].groupby("Embarked",as_index=False).mean()

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


In [ ]:
##描述性特征数值化
train_data_all["Embarked"]=train_data_all["Embarked"].map({"C":1,"Q":2,"S":3}).astype(int)
test_data_all["Embarked"]=test_data_all["Embarked"].map({"C":1,"Q":2,"S":3}).astype(int)

#### Age的分析处理

In [ ]:
#平均值插值
train_data_all["Age"].fillna(train_data_all["Age"].mean(),inplace=True)

#### 删除无用特征，并处理train和test

In [ ]:
###删除原始无用特征和中间分析过程产生的无用特征
train_data_all.drop(["PassengerId","Name","Ticket","Cabin"],axis=1,inplace=True)
test_data_all.drop(["PassengerId","Name","Ticket","Cabin"],axis=1,inplace=True)

In [ ]:
###训练样本，训练样本标签
X_train=train_data_all.drop("Survived",axis=1)
Y_train=train_data_all["Survived"]


#### 对数据进行归一化

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)

#### 划分训练集和测试集

In [ ]:
## 划分训练集和测试集
X_train,X_test,Y_train,Y_test=train_test_split(X_train,Y_train,test_size=0.3,random_state=0)

### 下面我们进入模型的训练

#### XGboost

In [ ]:
## 采用gridsearch方法搜寻最优的参数
param_grid = [
{'learning_rate': [0.05,0.1,0.2],'n_estimators': [15,20,25,30], 'max_depth':[15,20,25,30] , 'gamma':[0.2,0.4,0.6,0.8]}
]
xgb_model = xgb.XGBRegressor()
grid_search = GridSearchCV(xgb_model,param_grid,scoring='neg_mean_squared_error')
grid_search.fit(X_train,Y_train)
print(grid_search.best_params_)

{'gamma': 0.6, 'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 30}


In [ ]:
xgb_model = xgb.XGBRegressor(learning_rate=0.05,max_depth=15,n_estimators=300,gamma=0.6)
xgb_model.fit(X_train,Y_train)
y_pred = xgb_model.predict(X_test)
mse_test=mean_squared_error(Y_test,y_pred)
mse_train=mean_squared_error(Y_train,xgb_model.predict(X_train))
print(1-np.sqrt(mse_train))
print(1-np.sqrt(mse_test))

0.7051360995672302
0.6431346321783195


#### 随机森林

In [ ]:
## 采用gridsearch方法搜寻最优的参数
param_grid = [
{'n_estimators': [15,20,25,30], 'max_depth':[15,20,25,30] , 'random_state':[20,25,30,35,40]}
]
rfm = RandomForestClassifier()
grid_search = GridSearchCV(rfm,param_grid,n_jobs=-1,scoring='neg_mean_squared_error')
grid_search.fit(X_train,Y_train)
print(grid_search.best_params_)

{'max_depth': 15, 'n_estimators': 15, 'random_state': 20}


In [ ]:
### 随机森林
rfmodel = RandomForestClassifier(max_depth=15,n_estimators=15,random_state=20)    #实例化 
rfc = rfmodel.fit(X_train,Y_train)                      #用训练集数据训练模型
result1 = rfc.score(X_train,Y_train)
result2 = rfc.score(X_test,Y_test)                   #导入测试集，从接口中调用需要的信息
print(result1)
print(result2)

0.9678972712680578
0.8283582089552238
